In [ ]:
# ============================================================
# Inverse Shear Calculator — auto-detect cluster count from peaks.csv
# 支持 NFW 或 SIS；当 peaks.csv 为空/缺失时，自动按“无 cluster”处理（g=0）
# ============================================================

import numpy as np
import pandas as pd
import math
from dataclasses import dataclass
from typing import List, Tuple
from pathlib import Path
import os

# -------------------------
# 0) 配置区（按需修改）
# -------------------------
class CFG:
    # ---- 工作/路径 ----
    # ---- 工作/路径 ----
    BASE_DIR = Path("C:/Users/skyma/Desktop/t1")
    OUT_DIR  = BASE_DIR / "shear_out"
    INPUT_CSV  = BASE_DIR / "csv/sep4000px.csv"     # 观测(透镜后) catalog
    OUTPUT_CSV = OUT_DIR  / "sep4000px_w_shear.csv"   # 反剪切输出
    PEAKS_CSV  = BASE_DIR / "batch_sep/sep4000px_peaks.csv"  # cluster 峰值位置文件（允许无此文件）

    # ---- 成像/数值参数 ----
    PIXSCALE_ARCSEC = 0.263
    R_MIN_ARCSEC    = 1e-3        # 小半径数值保护
    G_MAX           = 0.95        # |g| 上限（稳定）

    # ---- 质量模型选择 ----
    MASS_MODEL = "SIS"            # "NFW" 或 "SIS"

    # ---- NFW 参数 ----
    M200C_MSUN = 8e14
    C200       = 4.0
    Z_LENS     = 0.3
    Z_SOURCE   = 1.0
    H0         = 70.0
    OMEGA_M    = 0.3
    OMEGA_L    = 1.0 - OMEGA_M

    # ---- SIS 参数 ----
    THETA_E_ARCSEC = 30.0
    CORE_ARCSEC    = 2.0

    # ---- 椭率约定与输出 ----
    ELLIP_CONVENTION  = "epsilon"    # 'epsilon' 或 'chi'
    WRITE_TO_NEW_COLS = True         # True: 写 e1_intrinsic/e2_intrinsic；False: 覆盖 e1/e2
    E_MAX_AFTER       = 0.95         # 反剪切后裁剪 |e| 上限；None 关闭
    ADD_GT_GX         = True         # 仅当 >=1 个 cluster 时才写 g_t/g_x

# -------------------------
# 1) 椭率/剪切 变换 & 逆变换
# -------------------------
def e12_to_complex(e1, e2): return e1 + 1j*e2
def complex_to_e12(z): return np.real(z), np.imag(z)

def epsilon_to_chi(eps):
    mod2 = np.abs(eps)**2
    return (2*eps) / (1 + mod2 + 1e-12)

def chi_to_epsilon(chi):
    mod = np.abs(chi)
    denom = 1 + np.sqrt(np.maximum(1 - mod**2, 0.0))
    denom = np.where(denom == 0, 1e-12, denom)
    return chi / denom

def lens_inverse_epsilon(e1_obs, e2_obs, g1, g2):
    eps_o = e12_to_complex(e1_obs, e2_obs)
    g     = e12_to_complex(g1, g2)
    den = 1 - np.conjugate(g)*eps_o
    den = np.where(np.abs(den)==0, 1e-12+0j, den)
    eps_s = (eps_o - g) / den
    return complex_to_e12(eps_s)

def lens_inverse_chi(e1_obs, e2_obs, g1, g2):
    chi_o = e12_to_complex(e1_obs, e2_obs)
    eps_o = chi_to_epsilon(chi_o)
    e1s, e2s = lens_inverse_epsilon(np.real(eps_o), np.imag(eps_o), g1, g2)
    chi_s = epsilon_to_chi(e1s + 1j*e2s)
    return complex_to_e12(chi_s)

def tangential_cross(g1, g2, dx, dy):
    phi = np.arctan2(dy, dx)
    c2, s2 = np.cos(2*phi), np.sin(2*phi)
    g_t = -(g1*c2 + g2*s2)
    g_x =  (g1*s2 - g2*c2)
    return g_t, g_x

# -------------------------
# 2) 读 peaks.csv（允许 0 个 cluster）
# -------------------------
def load_peaks_csv_allow_empty(path: Path) -> pd.DataFrame:
    """
    读取 peaks.csv（至少包含 x_peak,y_peak 列）。
    - 文件不存在 / 为空 / 列缺失：返回空 DataFrame（0 个 cluster）
    - 正常：返回 DataFrame，附加 cluster_id（1..N）
    """
    try:
        if (path is None) or (not Path(path).exists()):
            return pd.DataFrame(columns=["cluster_id","x_peak","y_peak"])
        peaks = pd.read_csv(path)
        if not {"x_peak","y_peak"}.issubset(peaks.columns):
            return pd.DataFrame(columns=["cluster_id","x_peak","y_peak"])
        peaks = peaks[["x_peak","y_peak"]].copy()
        peaks = peaks.dropna()
        if len(peaks)==0:
            return pd.DataFrame(columns=["cluster_id","x_peak","y_peak"])
        peaks.insert(0, "cluster_id", np.arange(1, len(peaks)+1, dtype=int))
        return peaks
    except Exception:
        # 任何读入异常 → 视作无 peaks
        return pd.DataFrame(columns=["cluster_id","x_peak","y_peak"])

# -------------------------
# 3) 宇宙学与 NFW 解析
# -------------------------
C_LIGHT = 299792.458               # km/s
G_MPC   = 4.30091e-9               # (Mpc km^2 / s^2) / Msun

def Ez(z, Om, Ol): return np.sqrt(Om*(1+z)**3 + Ol)

def comoving_distance_Mpc(z, H0, Om, Ol, n=2048):
    if z <= 0: return 0.0
    zs = np.linspace(0.0, float(z), n+1)
    integrand = 1.0 / Ez(zs, Om, Ol)
    dz = float(z) / n
    S = integrand[0] + integrand[-1] + 4*integrand[1:-1:2].sum() + 2*integrand[2:-2:2].sum()
    return (C_LIGHT / H0) * (dz/3.0) * S

def angular_diameter_distance_Mpc(z1, z2, H0, Om, Ol):
    if z2 <= z1: return 0.0
    chi2 = comoving_distance_Mpc(z2, H0, Om, Ol)
    chi1 = comoving_distance_Mpc(z1, H0, Om, Ol)
    return (chi2 - chi1) / (1 + z2)

def sigma_crit_Msun_per_Mpc2(zl, zs, H0, Om, Ol):
    Dd = angular_diameter_distance_Mpc(0.0, zl, H0, Om, Ol)
    Ds = angular_diameter_distance_Mpc(0.0, zs, H0, Om, Ol)
    Dds= angular_diameter_distance_Mpc(zl, zs, H0, Om, Ol)
    if Dd<=0 or Ds<=0 or Dds<=0: return np.inf
    return (C_LIGHT**2 / (4*np.pi*G_MPC)) * (Ds / (Dd * Dds))

def rho_c_crit_Msun_per_Mpc3(z, H0, Om, Ol):
    Hz = H0 * Ez(z, Om, Ol)
    return 3*(Hz**2) / (8*np.pi*G_MPC)

def nfw_scale_params(M200c, c200, z, H0, Om, Ol):
    rho_c = rho_c_crit_Msun_per_Mpc3(z, H0, Om, Ol)
    r200c = (3*M200c/(4*np.pi*200*rho_c))**(1/3)     # Mpc
    rs = r200c / c200
    delta_c = (200.0/3.0) * (c200**3) / (np.log(1+c200) - c200/(1+c200))
    rho_s = delta_c * rho_c
    return r200c, rs, rho_s

def _F_kappa(x):
    x = np.asarray(x, dtype=float)
    out = np.zeros_like(x)
    xm1 = x - 1.0
    mask_lt = x < (1 - 1e-6)
    mask_eq = np.abs(xm1) <= 1e-6
    mask_gt = x > (1 + 1e-6)
    if np.any(mask_lt):
        xx = x[mask_lt]
        sq = np.sqrt(1.0 - xx**2)
        at = np.arctanh(np.clip(np.sqrt((1-xx)/(1+xx)), 0, 1-1e-12))
        out[mask_lt] = (1 - 2*at/sq) / (xx**2 - 1)
    out[mask_eq] = 1.0/3.0
    if np.any(mask_gt):
        xx = x[mask_gt]
        sq = np.sqrt(xx**2 - 1.0)
        at = np.arctan(np.sqrt((xx-1)/(1+xx)))
        out[mask_gt] = (1 - 2*at/sq) / (xx**2 - 1)
    return out

def _G_bar(x):
    x = np.asarray(x, dtype=float)
    out = np.zeros_like(x)
    xm1 = x - 1.0
    mask_lt = x < (1 - 1e-6)
    mask_eq = np.abs(xm1) <= 1e-6
    mask_gt = x > (1 + 1e-6)
    if np.any(mask_lt):
        xx = x[mask_lt]
        term = np.arctanh(np.clip(np.sqrt((1-xx)/(1+xx)), 0, 1-1e-12))
        out[mask_lt] = np.log(xx/2.0) + 2*term/np.sqrt(1-xx**2)
    out[mask_eq] = 1 + np.log(0.5)
    if np.any(mask_gt):
        xx = x[mask_gt]
        term = np.arctan(np.sqrt((xx-1)/(1+xx)))
        out[mask_gt] = np.log(xx/2.0) + 2*term/np.sqrt(xx**2 - 1)
    return out

def nfw_kappa_gamma_t(R_phys_Mpc, rs_Mpc, kappa_s):
    x = np.maximum(R_phys_Mpc/rs_Mpc, 1e-12)
    f = _F_kappa(x)
    G = _G_bar(x)
    kappa = 2.0 * kappa_s * f
    kappa_bar = 4.0 * kappa_s * G / (x**2)
    gamma_t = kappa_bar - kappa
    return kappa, gamma_t

# -------------------------
# 4) 剪切场：NFW（严谨叠加） & SIS（严谨）
# -------------------------
def shear_field_NFW_rigorous(xs, ys, peaks_xy, cfg: CFG):
    xs = np.asarray(xs); ys = np.asarray(ys)
    # 预计算 NFW 尺度参数
    sigcrit = sigma_crit_Msun_per_Mpc2(cfg.Z_LENS, cfg.Z_SOURCE, cfg.H0, cfg.OMEGA_M, cfg.OMEGA_L)
    r200c, rs, rho_s = nfw_scale_params(cfg.M200C_MSUN, cfg.C200, cfg.Z_LENS, cfg.H0, cfg.OMEGA_M, cfg.OMEGA_L)
    kappa_s = (rho_s * rs) / sigcrit
    Dd = angular_diameter_distance_Mpc(0.0, cfg.Z_LENS, cfg.H0, cfg.OMEGA_M, cfg.OMEGA_L)

    kappa_tot = np.zeros_like(xs, dtype=float)
    gamma_c_tot = np.zeros_like(xs, dtype=complex)

    for (x0, y0) in peaks_xy:
        dx = xs - x0; dy = ys - y0
        R_pix = np.hypot(dx, dy)
        R_arcsec = np.sqrt((R_pix*cfg.PIXSCALE_ARCSEC)**2 + cfg.R_MIN_ARCSEC**2)
        theta_rad = R_arcsec * (np.pi / (180.0*3600.0))
        R_phys = Dd * theta_rad
        kappa_i, gamma_t_i = nfw_kappa_gamma_t(R_phys, rs, kappa_s)

        phi = np.arctan2(dy, dx)
        gamma_c_tot += - gamma_t_i * np.exp(2j*phi)  # 自旋-2切向
        kappa_tot   +=   kappa_i

    denom = np.maximum(1 - kappa_tot, 1e-6)
    g_c = gamma_c_tot / denom
    mod = np.abs(g_c)
    m = mod > cfg.G_MAX
    if np.any(m): g_c[m] *= (cfg.G_MAX / mod[m])
    return np.real(g_c), np.imag(g_c)

def shear_field_SIS_rigorous(xs, ys, peaks_xy, cfg: CFG):
    xs = np.asarray(xs); ys = np.asarray(ys)
    g1 = np.zeros_like(xs); g2 = np.zeros_like(ys)
    for (x0, y0) in peaks_xy:
        dx = xs - x0; dy = ys - y0
        R_pix = np.hypot(dx, dy)
        R_arc = np.sqrt((R_pix*cfg.PIXSCALE_ARCSEC)**2 + cfg.CORE_ARCSEC**2)
        kappa = 0.5 * cfg.THETA_E_ARCSEC / np.maximum(R_arc, 1e-9)
        gamma = kappa
        g_mod = gamma / np.maximum(1 - kappa, 1e-6)
        g_mod = np.clip(g_mod, 0.0, cfg.G_MAX)
        phi = np.arctan2(dy, dx)
        g1 += - g_mod * np.cos(2*phi)
        g2 += - g_mod * np.sin(2*phi)
    return g1, g2

# -------------------------
# 5) 反剪切写回 DataFrame
# -------------------------
def apply_inverse_reduced_shear_to_df(df, g1, g2, e1_col="e1", e2_col="e2",
                                      convention="epsilon", write_to_new_cols=True, suffix="_intrinsic", e_max=None):
    e1o = df[e1_col].to_numpy(); e2o = df[e2_col].to_numpy()
    if convention == "epsilon":
        e1s, e2s = lens_inverse_epsilon(e1o, e2o, g1, g2)
    elif convention == "chi":
        e1s, e2s = lens_inverse_chi(e1o, e2o, g1, g2)
    else:
        raise ValueError("convention must be 'epsilon' or 'chi'")

    if e_max is not None:
        e_mod = np.sqrt(e1s**2 + e2s**2)
        m = e_mod > e_max
        if np.any(m):
            scale = e_max / (e_mod[m] + 1e-12)
            e1s[m] *= scale; e2s[m] *= scale

    if write_to_new_cols:
        df[e1_col + suffix] = e1s
        df[e2_col + suffix] = e2s
    else:
        df[e1_col] = e1s
        df[e2_col] = e2s
    return df

# -------------------------
# 6) 主流程
# -------------------------
def main(cfg: CFG):
    Path(cfg.OUT_DIR).mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(cfg.INPUT_CSV)
    need = {"x","y","e1","e2"}
    if not need.issubset(df.columns):
        raise ValueError(f"{cfg.INPUT_CSV} 缺少必要列: {need}")

    # 自动读取 peaks.csv（允许 0 个）
    peaks_df = load_peaks_csv_allow_empty(cfg.PEAKS_CSV)
    n_peaks = len(peaks_df)
    print(f"[peaks] detected {n_peaks} cluster(s).")

    if n_peaks == 0:
        # 无 cluster：g=0，反剪切后即为本征（=观测）
        print("[info] No clusters found. Using g=0 everywhere.")
        g1 = np.zeros(len(df), dtype=float)
        g2 = np.zeros(len(df), dtype=float)
    else:
        peaks_xy = list(zip(peaks_df["x_peak"].to_numpy(), peaks_df["y_peak"].to_numpy()))
        if cfg.MASS_MODEL.upper() == "NFW":
            g1, g2 = shear_field_NFW_rigorous(
                xs=df["x"].to_numpy(), ys=df["y"].to_numpy(),
                peaks_xy=peaks_xy, cfg=cfg
            )
        elif cfg.MASS_MODEL.upper() == "SIS":
            g1, g2 = shear_field_SIS_rigorous(
                xs=df["x"].to_numpy(), ys=df["y"].to_numpy(),
                peaks_xy=peaks_xy, cfg=cfg
            )
        else:
            raise ValueError("MASS_MODEL 必须为 'NFW' 或 'SIS'")

    # 反剪切
    df = apply_inverse_reduced_shear_to_df(
        df, g1, g2,
        e1_col="e1", e2_col="e2",
        convention=cfg.ELLIP_CONVENTION,
        write_to_new_cols=cfg.WRITE_TO_NEW_COLS,
        suffix="_intrinsic",
        e_max=cfg.E_MAX_AFTER
    )

    # 可选：只有有 cluster 时才计算 g_t/g_x（以第一个峰为中心）
    if cfg.ADD_GT_GX and n_peaks > 0:
        x0, y0 = peaks_df.loc[0, "x_peak"], peaks_df.loc[0, "y_peak"]
        gt, gx = tangential_cross(g1, g2, df["x"].to_numpy()-x0, df["y"].to_numpy()-y0)
        df["g_t"] = gt; df["g_x"] = gx


    base, ext = os.path.splitext(cfg.OUTPUT_CSV)
    output_path = f"{base}_{cfg.MASS_MODEL}{ext}"
    df.to_csv(output_path, index=False, float_format="%.9f")
    print(f"[OK] saved -> {cfg.OUTPUT_CSV}")

# 运行
if __name__ == "__main__":
    main(CFG)


[peaks] detected 2 cluster(s).
[OK] saved -> C:\Users\skyma\Desktop\t1\shear_out\sep4000px_w_shear.csv
